# 从零实现BERT

在此项目中，我将会演示如何从头拆解BERT，重点关注了Transformer架构的核心概念、注意力机制和矩阵运算。BERT（双向编码器表示来自Transformers）是NLP中的一个关键模型，它使用双向方法来理解句子中单词的上下文。

我还将演示如何直接从Hugging Face提供的预训练BERT模型中加载张量。在运行此文件之前，你需要下载模型权重。以下是下载权重的官方链接：[Hugging Face BERT](https://huggingface.co/bert-base-uncased)

<div>
    <img src="images/all-steps.png"/ width=800>
</div>

## 模型初始化与输入处理

### 分词

我不会亲自在这里实现一个BPE分词器，但Andrej Karpathy有一个简洁的实现，如果感兴趣的话可以通过这个链接查看他的展示：https://github.com/karpathy/minbpe

In [2]:
# 模型加载
import torch
from transformers import BertForMaskedLM, BertTokenizer

model_path="bert-base-uncased"

tokenizer=BertTokenizer.from_pretrained(model_path)

In [3]:
tokenizer.encode("hello world!")

[101, 7592, 2088, 999, 102]

In [4]:
tokenizer.decode(tokenizer.encode("hello world!"))

'[CLS] hello world! [SEP]'

### 加载和检查模型配置与权重

在此步骤中，模型的配置和权重会从预训练模型文件中加载。此步骤确保在进行前向传播时，必要的参数已经准备就绪。

通常，读取模型文件依赖于模型类的编写方式以及其中的变量名。然而，由于我们是从零开始实现BERT模型，我们将逐个读取张量，仔细检查BERT架构中的嵌入层、注意力头和前馈网络。

In [5]:
# 1. 使用 transformers 加载预训练模型

# 加载模型配置和权重
model = BertForMaskedLM.from_pretrained(model_path, torch_dtype=torch.float32)

Some weights of the model checkpoint at /home/jlpanc/anaconda3/envs/codef/STA/bert-base-uncased were not used when initializing BertForMaskedLM: ['bert.pooler.dense.bias', 'bert.pooler.dense.weight', 'cls.seq_relationship.bias', 'cls.seq_relationship.weight']
- This IS expected if you are initializing BertForMaskedLM from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForMaskedLM from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [6]:
model

BertForMaskedLM(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(30522, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0-11): 12 x BertLayer(
          (attention): BertAttention(
            (self): BertSdpaSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12, elementwi

In [7]:
model = model.state_dict()

In [8]:
model.keys()

odict_keys(['bert.embeddings.word_embeddings.weight', 'bert.embeddings.position_embeddings.weight', 'bert.embeddings.token_type_embeddings.weight', 'bert.embeddings.LayerNorm.weight', 'bert.embeddings.LayerNorm.bias', 'bert.encoder.layer.0.attention.self.query.weight', 'bert.encoder.layer.0.attention.self.query.bias', 'bert.encoder.layer.0.attention.self.key.weight', 'bert.encoder.layer.0.attention.self.key.bias', 'bert.encoder.layer.0.attention.self.value.weight', 'bert.encoder.layer.0.attention.self.value.bias', 'bert.encoder.layer.0.attention.output.dense.weight', 'bert.encoder.layer.0.attention.output.dense.bias', 'bert.encoder.layer.0.attention.output.LayerNorm.weight', 'bert.encoder.layer.0.attention.output.LayerNorm.bias', 'bert.encoder.layer.0.intermediate.dense.weight', 'bert.encoder.layer.0.intermediate.dense.bias', 'bert.encoder.layer.0.output.dense.weight', 'bert.encoder.layer.0.output.dense.bias', 'bert.encoder.layer.0.output.LayerNorm.weight', 'bert.encoder.layer.0.output

In [9]:
# float32
model['bert.embeddings.word_embeddings.weight'].shape

torch.Size([30522, 768])

In [10]:
import json
with open("/home/jlpanc/anaconda3/envs/codef/STA/bert-base-uncased/config.json", "r") as f:
    config = json.load(f)
config

{'architectures': ['BertForMaskedLM'],
 'attention_probs_dropout_prob': 0.1,
 'gradient_checkpointing': False,
 'hidden_act': 'gelu',
 'hidden_dropout_prob': 0.1,
 'hidden_size': 768,
 'initializer_range': 0.02,
 'intermediate_size': 3072,
 'layer_norm_eps': 1e-12,
 'max_position_embeddings': 512,
 'model_type': 'bert',
 'num_attention_heads': 12,
 'num_hidden_layers': 12,
 'pad_token_id': 0,
 'position_embedding_type': 'absolute',
 'transformers_version': '4.6.0.dev0',
 'type_vocab_size': 2,
 'use_cache': True,
 'vocab_size': 30522}

### 
我们使用该配置推断模型的详细信息，如下所示：

	1.	该模型有12个Transformer层。
	2.	每个多头注意力模块有12个头。
	3.	词汇表大小为30,522个token。
	4.	隐藏层的维度为768维。
	5.	前馈层的中间层大小为3072维。

In [ ]:
dim = config["hidden_size"]
n_layers = config["num_hidden_layers"]
n_heads = config["num_attention_heads"]
vocab_size = config["vocab_size"]
norm_eps = config["layer_norm_eps"]
eps = config["layer_norm_eps"]

### 将文本转换为Token
在这里，我们使用 tiktoken 库（由 OpenAI 开发）作为分词器。
<div>
    <img src="images/embedding_layers.png" width="600"/>
</div>

In [12]:
# 1. 使用BERT分词器将提示转换为token ID。
prompt = "When in Rome, do as the [MASK] do."
input_ids = tokenizer.encode(prompt, return_tensors='pt')
print("input_ids: ", input_ids)
# 2. 创建一个token类型ID的张量（全为零，因为我们只有一个句子）。
token_type_ids = torch.zeros_like(input_ids)
print("token_type_ids: ", token_type_ids)
# 3. 将input_ids转换为张量。
input_ids = torch.tensor(input_ids[0])
# 4. 计算并打印输入提示中的token数量。
q_len = len(input_ids)
print("q_len: ", q_len)

input_ids:  tensor([[ 101, 2043, 1999, 4199, 1010, 2079, 2004, 1996,  103, 2079, 1012,  102]])
token_type_ids:  tensor([[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]])
q_len:  12


/tmp/ipykernel_549748/3428054749.py:9: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  input_ids = torch.tensor(input_ids[0])


### 生成Token嵌入

在本节中，我们使用BERT的预训练嵌入层将输入的token转换为对应的嵌入。虽然BERT使用内置的神经网络模块来完成这一过程，但理解这种转换至关重要。

因此，我们的[10x1] token现在被转换为[10x768]，也就是说，12个token（每个token一个）的嵌入长度为768。

注意：在整个过程中请注意张量的形状变化，这将有助于更容易理解整个架构。

<div>
    <img src="images/embeddings_output.png" width="600"/>
</div>

In [15]:
# 1. 从预训练的BERT模型中加载词嵌入
embedding_layer = torch.nn.Embedding.from_pretrained(model['bert.embeddings.word_embeddings.weight'])
# 2. 从预训练的BERT模型中加载位置嵌入
position_embeddings = torch.nn.Embedding.from_pretrained(model['bert.embeddings.position_embeddings.weight'])
# 3. 从预训练的BERT模型中加载分段（token类型）嵌入
segment_embeddings = torch.nn.Embedding.from_pretrained(model['bert.embeddings.token_type_embeddings.weight'])
# embedding_layer.weight.data.copy_(model["model.embed_tokens.weight"])
token_embeddings_unnormalized = embedding_layer(input_ids)

In [14]:
token_embeddings_unnormalized

tensor([[ 0.0136, -0.0265, -0.0235,  ...,  0.0087,  0.0071,  0.0151],
        [-0.0463,  0.0259, -0.0240,  ..., -0.0416,  0.0289,  0.0316],
        [-0.0278,  0.0039, -0.0035,  ...,  0.0078, -0.0306,  0.0221],
        ...,
        [ 0.0066, -0.0533,  0.0057,  ..., -0.0140, -0.0522, -0.0088],
        [-0.0207, -0.0020, -0.0118,  ...,  0.0128,  0.0200,  0.0259],
        [-0.0145, -0.0100,  0.0060,  ..., -0.0250,  0.0046, -0.0015]])

In [16]:
token_embeddings_unnormalized.shape

torch.Size([12, 768])

In [21]:
position_ids = torch.arange(q_len, dtype=torch.long).unsqueeze(0)
position_embeds = position_embeddings(position_ids)
position_embeds.shape

torch.Size([1, 12, 768])

In [25]:
segment_embeds = segment_embeddings(token_type_ids)
segment_embeds.shape

torch.Size([1, 12, 768])

In [26]:
embeddings_unnormalized = token_embeddings_unnormalized+position_embeds+segment_embeds
embeddings_unnormalized.shape

torch.Size([1, 12, 768])

In [27]:
embeddings_unnormalized

tensor([[[ 0.0316, -0.0411, -0.0564,  ...,  0.0021,  0.0044,  0.0219],
         [-0.0381,  0.0392, -0.0397,  ..., -0.0193,  0.0553,  0.0177],
         [-0.0387,  0.0129, -0.0114,  ...,  0.0161, -0.0153,  0.0062],
         ...,
         [ 0.0135, -0.0457, -0.0071,  ...,  0.0045, -0.0567, -0.0130],
         [-0.0299,  0.0069, -0.0199,  ...,  0.0310,  0.0295,  0.0295],
         [-0.0253,  0.0052, -0.0068,  ..., -0.0129,  0.0153, -0.0015]]])

### 应用层归一化  
在BERT中，我们在嵌入层和其他阶段之后应用层归一化。  
提示：此步骤不会改变张量的形状，它只是对隐藏维度中的值进行归一化。

### 需要额外注意的事项：
1. 层归一化通过对特征进行归一化，防止内部协变量偏移。
2. 为了数值稳定性（防止除以零），会添加一个小的epsilon值（从配置中获得）。

### 构建Transformer的第一层

#### 归一化
在BERT中，我们首先对每个Transformer层的输入应用层归一化。

你会注意到，我从模型字典中访问`encoder.layer.0`（这对应于BERT的第一层）。  
应用归一化后，张量的形状仍然与输入相同，为[12x768]，但数值现在已归一化。
<div>
    <img src="images/layer_norm.png" width="600"/>
</div>

In [29]:
def layer_norm(x, gamma, beta, eps=1e-12):
    """
    参数:
    x (torch.Tensor): 输入张量，形状为 (batch_size, num_features)
    gamma (torch.Tensor): 缩放参数，形状与x的最后一个维度相同
    beta (torch.Tensor): 偏移参数，形状与x的最后一个维度相同
    eps (float): 用于数值稳定性的常量（防止除以零）
    
    返回:
    torch.Tensor: 经过LayerNorm后的输出
    """
    # 计算均值和方差
    mean = x.mean(dim=-1, keepdim=True)
    variance = x.var(dim=-1, keepdim=True, unbiased=False)
    
    # 标准化
    x_normalized = (x - mean) / torch.sqrt(variance + eps)
    
    # 缩放和偏移
    out = gamma * x_normalized + beta
    
    return out

# 构建第一个Transformer层  
### 归一化  
正如你所见，通过模型字典中的layer0后，输出的张量仍然是形状为[10*768]，但数值已经归一化了。

<div>
    <img src="images/embeding_norm1.png", width=500>
</div>


In [30]:
# 初始化 LayerNorm and Dropout
# layer_norm = torch.nn.LayerNorm(768, eps=1e-12)
# layer_norm.weight.data = model['bert.embeddings.LayerNorm.weight']
# layer_norm.bias.data = model['bert.embeddings.LayerNorm.bias']

# 执行层归一化
normalized_embeddings = layer_norm(embeddings_unnormalized, model['bert.embeddings.LayerNorm.weight'], model['bert.embeddings.LayerNorm.bias'], eps=1e-12)

# 应用Dropout
# final_embeddings = dropout(normalized_embeddings)

# 打印结果
print("Embeddings after LayerNorm:", normalized_embeddings)
# print("Final Embeddings:", final_embeddings)

Embeddings after LayerNorm: tensor([[[ 0.1686, -0.2858, -0.3261,  ..., -0.0276,  0.0383,  0.1640],
         [-0.4807,  0.8188, -0.4227,  ..., -0.1415,  1.1064,  0.5430],
         [-0.4992,  0.3891,  0.0274,  ...,  0.3810, -0.0397,  0.3439],
         ...,
         [ 0.5077, -0.5198,  0.1786,  ...,  0.2883, -0.6555,  0.0927],
         [-0.3585,  0.2777, -0.1210,  ...,  0.5949,  0.6856,  0.7453],
         [-0.4771,  0.0871, -0.0770,  ..., -0.2191,  0.3020,  0.0196]]])


In [31]:
normalized_embeddings.shape

torch.Size([1, 12, 768])

## 自注意力机制  
本节深入探讨了BERT中自注意力机制的实现，包括Query、Key和Value矩阵的计算，以及它们如何共同生成注意力分数。

In [32]:
# 从BERT模型中提取第一个注意力层的query、key和value的权重
q_layer0 = model["bert.encoder.layer.0.attention.self.query.weight"]
k_layer0 = model["bert.encoder.layer.0.attention.self.key.weight"]
v_layer0 = model["bert.encoder.layer.0.attention.self.value.weight"]
# 提取第一个注意力层中query、key和value的偏置
q_layer0_bias = model['bert.encoder.layer.0.attention.self.query.bias']
k_layer0_bias = model['bert.encoder.layer.0.attention.self.key.bias']
v_layer0_bias = model['bert.encoder.layer.0.attention.self.value.bias']

In [33]:
# 通过将归一化后的嵌入与对应的权重相乘并加上偏置，计算query、key和value状态
query_states = torch.matmul(normalized_embeddings, q_layer0.T)+q_layer0_bias
key_states = torch.matmul(normalized_embeddings, k_layer0.T)+k_layer0_bias
value_states = torch.matmul(normalized_embeddings, v_layer0.T)+v_layer0_bias

In [34]:
# 定义一个函数用于对状态进行重塑和排列，以适应多头注意力
def transpose_for_scores(x):
    new_x_shape = x.size()[:-1] + (12, 64) # (num_attention_heads, attention_head_size)
    x = x.view(new_x_shape) # Reshape the tensor to separate attention heads
    return x.permute(0, 2, 1, 3) # Permute the tensor dimensions for attention computation

## 多头注意力机制解释
多头注意力是BERT架构中的一个核心组件，它使模型能够同时关注输入序列的不同部分。与只计算一次注意力不同，多头注意力会多次执行注意力机制（并行进行），每次使用不同的线性变换对query、key和value向量进行投影。这一过程产生了多个“头”的注意力，每个头都专注于输入的不同方面。

在BERT中，每个注意力头独立运行，通过关注序列中不同位置的tokens，捕捉它们之间的多样关系。计算完每个头的注意力后，模型会将这些结果拼接起来，并通过线性变换生成最终的输出。这种方法通过融合多个注意力视角的信息，显著增强了BERT对上下文的理解能力，使其在自然语言处理任务中表现出色。

In [37]:
# 对 query、key 和 value 状态应用 transpose_for_scores 函数
transposed_query_states = transpose_for_scores(query_states)
transposed_key_states = transpose_for_scores(key_states)
transposed_value_states = transpose_for_scores(value_states)

In [39]:
print('transposed_query_states shape: ',transposed_query_states.shape)
print('transposed_key_states shape: ',transposed_key_states.shape)
print('transposed_value_states shape: ',transposed_value_states.shape)

transposed_query_states shape:  torch.Size([1, 12, 12, 64])
transposed_key_states shape:  torch.Size([1, 12, 12, 64])
transposed_value_states shape:  torch.Size([1, 12, 12, 64])


### 计算注意力得分

然后我们在自注意力机制中将查询（queries）和键（key）矩阵相乘：

	• 进行这个操作会产生一个得分，用来映射每个标记与序列中所有其他标记之间的关系。
	• 该得分表示每个标记的查询与每个其他标记的键对齐的程度。
	• 得出的注意力得分矩阵（称为 qk_per_token）的形状为 [12x12]，其中12表示输入序列中的标记数量。

#### 从零实现注意力机制

我们来加载transformer第一层的注意力头。

<div>
    <img src="images/qkv.png" width="600"/>
</div>

### 加载查询、键、值和输出矩阵

当我们从模型中加载查询、键、值和输出矩阵时，我们注意到它们的形状为：

	• 查询（Query）：[768x768]
	• 键（Key）：[768x768]
	• 值（Value）：[768x768]
	• 输出（Output）：[768x768]

乍一看，这可能显得有些不寻常，因为理想情况下，我们希望为每个注意力头分别拥有独立的q、k、v和o矩阵。

然而，BERT的作者为了提高效率，将这些矩阵捆绑在一起，这样可以并行化注意力头的计算。

现在，让我们展开这些捆绑的矩阵，以便分别访问每个注意力头。

In [40]:
# 使用缩放点积注意力（内置函数）计算注意力输出
attn_output = torch.nn.functional.scaled_dot_product_attention(
    t_query_states,
    t_key_states,
    t_value_states,
    attn_mask=None,
    dropout_p= 0.0,
# q_len > 1 是必要的，因为当 q_len == 1 时，AttentionMaskConverter.to_causal_4d 不会创建因果遮罩。
    is_causal= False,
)

In [45]:
attn_output.shape

torch.Size([1, 12, 768])

In [41]:
# 重塑并转置注意力输出以匹配预期的维度
attn_output = attn_output.transpose(1, 2).reshape(1, 12, 768)

In [42]:
# 提取第一层注意力层的输出全连接层的权重和偏置
o_layer0 = model["bert.encoder.layer.0.attention.output.dense.weight"]
o_layer0_bias = model['bert.encoder.layer.0.attention.output.dense.bias']
# 通过应用全连接层变换计算最终的注意力输出状态
ouput_states = torch.matmul(attn_output, o_layer0.T)+o_layer0_bias

In [43]:
ouput_states

tensor([[[ 0.0469,  0.1639, -0.0603,  ..., -0.1527,  0.0335,  0.1802],
         [ 0.1023,  0.3846, -0.0411,  ..., -0.5442, -0.0701, -0.0260],
         [-0.0404,  0.2899,  0.1311,  ..., -0.2009, -0.2455, -0.0298],
         ...,
         [ 0.2090,  0.3050,  0.0050,  ..., -0.2973, -0.1212, -0.0459],
         [ 0.0831, -0.0208, -0.1554,  ..., -0.4028, -0.2934, -0.0784],
         [ 0.3431, -0.1718,  0.0026,  ..., -0.3566, -0.2113,  0.0350]]])

In [44]:
ouput_states.shape

torch.Size([1, 12, 768])

In [46]:
# 提取第一层注意力层输出的LayerNorm权重和偏置
layer_0_output_norm_weight = model['bert.encoder.layer.0.attention.output.LayerNorm.weight']
layer_0_output_norm_bias = model['bert.encoder.layer.0.attention.output.LayerNorm.bias']
# 将LayerNorm应用于与输入嵌入结合的输出状态
attention_output = torch.nn.functional.layer_norm(ouput_states+normalized_embeddings, [ouput_states.size(-1)], layer_0_output_norm_weight, layer_0_output_norm_bias)

In [48]:
attention_output.shape

torch.Size([1, 12, 768])

## 上采样

在像BERT这样的Transformer模型中，上采样指的是通过增加表示的分辨率或维度的技术，以确保模型能够在各层之间捕捉并保留更多的详细信息。尽管像双线性插值和转置卷积这样的传统上采样方法更多与图像处理任务相关，这一概念在NLP模型中也有类似的应用，即通过增加或保持嵌入的维度来丰富特征空间。这个过程对于确保模型能够保持较高的信息密度至关重要，使得后续层（如前馈神经网络）能够处理更加丰富和复杂的表示，最终提升模型理解和生成细腻文本的能力。

## 前馈神经网络

在自注意力机制之后，BERT的每一层Transformer都会应用一个前馈神经网络来进一步处理嵌入。这一步对于引入非线性特征非常关键，能够使模型从输入数据中学习到更复杂的表示。

In [49]:
# 提取第一层Transformer中间密集层的权重和偏置
layer_0_intermediate_dense_weight = model['bert.encoder.layer.0.intermediate.dense.weight']
layer_0_intermediate_dense_bias = model['bert.encoder.layer.0.intermediate.dense.bias']
# 计算中间密集层的输出
intermediate_dense_output = torch.matmul(attention_output, layer_0_intermediate_dense_weight.T)+layer_0_intermediate_dense_bias
intermediate_dense_output

tensor([[[-3.1348, -2.2779, -3.1132,  ..., -2.4296, -3.1535, -2.9147],
         [-1.7857, -1.4517, -3.6339,  ..., -1.9001, -2.6653,  0.6086],
         [-3.0365, -0.8983, -4.2322,  ..., -2.2538, -2.9196, -2.1696],
         ...,
         [-1.7389, -2.1817, -4.1068,  ..., -1.2588, -1.4242, -1.0926],
         [-2.9177, -1.5600, -3.8377,  ..., -1.5096, -2.2996, -1.7339],
         [-3.7864, -2.3012, -3.4133,  ..., -2.6182, -2.6048, -3.1074]]])

In [50]:
intermediate_dense_output.shape

torch.Size([1, 12, 3072])

# 激活函数

在中间密集层计算出输出后，激活函数被应用以引入非线性到模型中。在BERT中，使用了GELU（高斯误差线性单元）激活函数。与更常见的ReLU（线性整流单元）不同，GELU是一个平滑、连续的函数，它将输出建模为概率近似，允许输出包含正值和负值。这一特性帮助BERT更好地捕捉输入数据中的细微关系，从而在各种自然语言处理任务中实现更好的表现。

In [51]:
# 将GELU激活函数应用于中间密集层的输出
intermediate_output = torch.nn.functional.gelu(intermediate_dense_output)
intermediate_output

tensor([[[-2.6957e-03, -2.5889e-02, -2.8811e-03,  ..., -1.8364e-02,
          -2.5440e-03, -5.1889e-03],
         [-6.6208e-02, -1.0640e-01, -5.0749e-04,  ..., -5.4553e-02,
          -1.0250e-02,  4.4340e-01],
         [-3.6334e-03, -1.6575e-01, -4.8812e-05,  ..., -2.7281e-02,
          -5.1157e-03, -3.2584e-02],
         ...,
         [-7.1338e-02, -3.1777e-02, -8.2369e-05,  ..., -1.3097e-01,
          -1.0994e-01, -1.5000e-01],
         [-5.1445e-03, -9.2632e-02, -2.3813e-04,  ..., -9.8983e-02,
          -2.4687e-02, -7.1898e-02],
         [-2.8956e-04, -2.4598e-02, -1.0957e-03,  ..., -1.1571e-02,
          -1.1973e-02, -2.9320e-03]]])

In [52]:
intermediate_output.shape

torch.Size([1, 12, 3072])

## 下采样

在像BERT这样的Transformer模型中，下采样指的是降低表示的分辨率或维度的技术，以确保模型能够更高效地处理信息，同时保留最关键的特征。尽管传统的下采样方法如池化或步幅卷积更常见于图像处理任务，这一概念可以在NLP模型中松散地应用，即通过减少嵌入的维度来简化特征空间。这个过程对于管理模型的计算负载至关重要，它允许后续层（如前馈神经网络）专注于最相关的信息。通过压缩数据，下采样帮助模型在复杂性和效率之间保持平衡，从而增强其处理大规模输入序列和生成准确预测的能力。

In [53]:
layer_0_intermediate_dense_weight.shape

torch.Size([3072, 768])

In [54]:
# 提取第一层Transformer的输出密集层的权重和偏置
layer_0_output_dense_weight = model['bert.encoder.layer.0.output.dense.weight']
layer_0_output_dense_bias = model['bert.encoder.layer.0.output.dense.bias']
# 计算第一层Transformer的最终密集层输出
output_dense = torch.matmul(intermediate_output, layer_0_output_dense_weight.T)+layer_0_output_dense_bias
output_dense.shape

torch.Size([1, 12, 768])

In [56]:
# 提取第一层Transformer输出的LayerNorm权重和偏置
layer_0_output_layernorm_weight = model['bert.encoder.layer.0.output.LayerNorm.weight']
layer_0_output_layernorm_bias = model['bert.encoder.layer.0.output.LayerNorm.bias']
# 将LayerNorm应用于密集层输出与原始注意力输出之和
layer_norm(output_dense+attention_output, layer_0_output_layernorm_weight, layer_0_output_layernorm_bias, eps=1e-12).shape

torch.Size([1, 12, 768])

### 通过BERT的12层Transformer的前向传播

接下来，我们对BERT模型的12层进行前向传播。

每一层都应用多头自注意力机制来捕捉标记之间的关系，随后通过带有GELU激活函数的密集层引入非线性，并通过Layer Normalization来稳定输出。

每层的输出都会传递到下一层，经过所有12层的处理后，我们得到一个最终的上下文化嵌入，准备用于下游任务，如分类或问答等。

## 一次性全部处理
<div>
    <img src="images/12self_attention.png" width="600px"/>
</div>
没错，就是这样。我们之前做的所有事情，现在要一次性对每一层都进行处理。
<br>

In [57]:
# 使用输入层的归一化嵌入进行初始化
layer_embedding_norm = normalized_embeddings
# 遍历BERT中的所有12个Transformer层
for layer in range(12):
    # 提取当前层注意力机制的query、key和value的权重
    q_layer = model[f"bert.encoder.layer.{layer}.attention.self.query.weight"]
    k_layer = model[f"bert.encoder.layer.{layer}.attention.self.key.weight"]
    v_layer = model[f"bert.encoder.layer.{layer}.attention.self.value.weight"]
    
    # 提取query、key和value的偏置
    q_layer_bias = model[f'bert.encoder.layer.{layer}.attention.self.query.bias']
    k_layer_bias = model[f'bert.encoder.layer.{layer}.attention.self.key.bias']
    v_layer_bias = model[f'bert.encoder.layer.{layer}.attention.self.value.bias']
    
    # 通过应用相应的权重和偏置计算query、key和value状态
    query_states = torch.matmul(layer_embedding_norm, q_layer.T)+q_layer_bias
    key_states = torch.matmul(layer_embedding_norm, k_layer.T)+k_layer_bias
    value_states = torch.matmul(layer_embedding_norm, v_layer.T)+v_layer_bias
    
    # 转置并重塑状态以进行多头注意力计算
    t_query_states = transpose_for_scores(query_states)
    t_key_states = transpose_for_scores(key_states)
    t_value_states = transpose_for_scores(value_states)
    
    # 使用缩放点积注意力计算注意力输出
    attn_output = torch.nn.functional.scaled_dot_product_attention(
        t_query_states,
        t_key_states,
        t_value_states,
        attn_mask=None,
        dropout_p= 0.0,
        # q_len > 1 是必要的，因为当 q_len == 1 时，AttentionMaskConverter.to_causal_4d 不会创建因果遮罩。
        is_causal= False,
    )
    attn_output = attn_output.transpose(1, 2).reshape(1, 12, 768)
    
    # 提取注意力输出变换的密集层权重和偏置
    o_layer = model[f"bert.encoder.layer.{layer}.attention.output.dense.weight"]
    o_layer_bias = model[f'bert.encoder.layer.{layer}.attention.output.dense.bias']
    ouput_states = torch.matmul(attn_output, o_layer.T)+o_layer_bias
    
    # 提取注意力输出的LayerNorm权重和偏置
    layer_output_norm_weight = model[f'bert.encoder.layer.{layer}.attention.output.LayerNorm.weight']
    layer_output_norm_bias = model[f'bert.encoder.layer.{layer}.attention.output.LayerNorm.bias']
    attention_output11 = layer_norm(ouput_states+layer_embedding_norm, layer_output_norm_weight, layer_output_norm_bias, eps=1e-12)

    # 计算中间密集层的输出
    layer_intermediate_dense_weight = model[f'bert.encoder.layer.{layer}.intermediate.dense.weight']
    layer_intermediate_dense_bias = model[f'bert.encoder.layer.{layer}.intermediate.dense.bias']
    intermediate_dense_output = torch.matmul(attention_output11, layer_intermediate_dense_weight.T)+layer_intermediate_dense_bias
    
    # 应用GELU激活函数
    intermediate_output = torch.nn.functional.gelu(intermediate_dense_output)
    
    # 计算输出密集层的变换
    layer_output_dense_weight = model[f'bert.encoder.layer.{layer}.output.dense.weight']
    layer_output_dense_bias = model[f'bert.encoder.layer.{layer}.output.dense.bias']
    output_dense = torch.matmul(intermediate_output, layer_output_dense_weight.T)+layer_output_dense_bias
    
    # 为当前层应用最终的LayerNorm
    layer_output_layernorm_weight = model[f'bert.encoder.layer.{layer}.output.LayerNorm.weight']
    layer_output_layernorm_bias = model[f'bert.encoder.layer.{layer}.output.LayerNorm.bias']
    layer_embedding_norm = layer_norm(output_dense+attention_output11, layer_output_layernorm_weight, layer_output_layernorm_bias, eps=1e-12)


# 所有12层Transformer后的最终输出
    layer_embedding_norm

tensor([[[ 0.0698,  0.3857, -0.0319,  ..., -0.1526,  0.0446,  0.2253],
         [-0.5065,  0.7490,  0.5782,  ..., -0.7123, -0.4653, -0.7150],
         [-0.0510, -0.6486, -0.1926,  ..., -0.1028,  0.0320,  0.1524],
         ...,
         [ 0.8641,  0.3875,  0.5311,  ...,  0.2063,  0.1742,  0.1639],
         [-0.4663, -0.5197, -0.2011,  ...,  0.6513,  0.0255, -0.1058],
         [ 0.7993,  0.2471, -0.0632,  ...,  0.1179, -0.7094, -0.0537]]])

In [58]:
layer_embedding_norm.shape

torch.Size([1, 12, 768])

In [59]:
# 提取预测头中密集层的权重和偏置
layer_intermediate_dense_weight = model['cls.predictions.transform.dense.weight']
layer_intermediate_dense_bias = model['cls.predictions.transform.dense.bias']

# 计算预测头中密集层的输出
prediction_output = torch.matmul(layer_embedding_norm, layer_intermediate_dense_weight.T)+layer_intermediate_dense_bias
prediction_output

tensor([[[-0.9308,  0.5810, -1.5488,  ..., -0.3969, -0.2049, -0.7886],
         [-0.4524,  1.4434,  0.7572,  ...,  0.8591,  1.1784,  1.1476],
         [ 0.1031,  0.8686,  0.9038,  ...,  1.2142,  0.3628,  1.5648],
         ...,
         [ 1.0442, -0.6966,  1.6925,  ...,  1.5027,  0.0384,  2.1948],
         [-0.2648, -0.0293,  0.6970,  ...,  0.6369,  1.0992,  1.0511],
         [ 0.1265, -1.1419,  0.2915,  ..., -0.2913,  0.0908,  0.3325]]])

In [60]:
prediction_output.shape

torch.Size([1, 12, 768])

In [61]:
# 对预测输出应用GELU激活函数
prediction_act_dense_output = torch.nn.functional.gelu(prediction_output)

# 提取预测输出的LayerNorm权重和偏置
prediction_layernorm_weight = model['cls.predictions.transform.LayerNorm.weight']
prediction_layernorm_bias = model['cls.predictions.transform.LayerNorm.bias']
# 将LayerNorm应用于激活后的密集层输出
prediction_dense_output = layer_norm(prediction_act_dense_output, prediction_layernorm_weight, prediction_layernorm_bias, eps=1e-12)
prediction_dense_output

tensor([[[-0.6974,  1.0231, -0.0350,  ..., -0.3549, -0.0176,  0.0247],
         [-4.3757,  1.5879, -1.3361,  ..., -1.1796,  0.3573,  0.4541],
         [-3.5006, -0.2887, -0.2252,  ...,  0.9825, -2.4059,  3.3587],
         ...,
         [-0.4326, -4.0172,  2.9924,  ...,  1.8936, -3.3367,  5.7582],
         [-2.9313, -2.1162, -0.0482,  ..., -0.5154,  1.6312,  1.7295],
         [-0.8187, -2.2804,  0.7585,  ..., -2.2679, -0.4951,  1.3419]]])

In [62]:
prediction_dense_output.shape

torch.Size([1, 12, 768])

In [63]:
# 提取用于最终预测的解码器权重和偏置
decoder_weight = model['cls.predictions.decoder.weight'] # 形状 [vocab_size, hidden_size]
decoder_bias = model['cls.predictions.decoder.bias'] # 形状 [vocab_size]

# 提取用于最终预测的解码器权重和偏置
# if 'cls.predictions.bias' in model:
#     decoder_bias += model['cls.predictions.bias']

# 将最终输出投影到词汇空间以计算logits
logits = torch.matmul(prediction_dense_output, decoder_weight.T) + decoder_bias

In [64]:
logits.shape

torch.Size([1, 12, 30522])

In [65]:
# 获取预测的标记 (词汇表中的索引)
predictions = torch.argmax(logits, dim=-1)

In [66]:
predictions.shape

torch.Size([1, 12])

In [67]:
input_ids = tokenizer.encode(prompt, return_tensors='pt')

In [68]:
input_ids

tensor([[ 101, 2043, 1999, 4199, 1010, 2079, 2004, 1996,  103, 2079, 1012,  102]])

## lets go！

In [69]:
tokenizer.decode(predictions[0], return_tensors='pt')

'. when in rome, do as the romans do..'

# thank you

结束了！希望你喜欢阅读这一切！